### 법령정보포털 api 판례 파싱 및 저장

In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import trange
from multiprocessing import Pool, cpu_count

def fetch_case_details(link):
    try:
        detail_response = urlopen("https://www.law.go.kr" + link).read()
        detail_xtree = ET.fromstring(detail_response)

        return {
            '판례정보일련번호': detail_xtree.find('판례정보일련번호').text,
            '사건명': detail_xtree.find('사건명').text,
            '사건번호': detail_xtree.find('사건번호').text,
            '선고일자': detail_xtree.find('선고일자').text,
            '법원명': detail_xtree.find('법원명').text,
            '법원종류코드': detail_xtree.find('법원종류코드').text,
            '사건종류명': detail_xtree.find('사건종류명').text,
            '사건종류코드': detail_xtree.find('사건종류코드').text,
            '판시사항': detail_xtree.find('판시사항').text,
            '판결요지': detail_xtree.find('판결요지').text,
            '판례내용': detail_xtree.find('판례내용').text
        }
    except Exception as e:
        print(f"Error fetching details for link {link}: {e}")
        return None

def fetch_page(page):
    url = f"https://www.law.go.kr/DRF/lawSearch.do?OC=mu07010&target=prec&type=XML&page={page}"
    response = urlopen(url).read()
    xtree = ET.fromstring(response)
    items = xtree[5:]
    links = [node.find('판례상세링크').text.replace('HTML', 'XML') for node in items]
    return links

if __name__ == "__main__":
    url = "https://www.law.go.kr/DRF/lawSearch.do?OC=mu07010&target=prec&type=XML"
    response = urlopen(url).read()
    xtree = ET.fromstring(response)

    totalCnt = int(xtree.find('totalCnt').text)
    total_pages = (totalCnt // 20) + 1

    # Fetch all case detail links using multiprocessing
    with Pool(8) as pool:
        all_links = []
        for links in pool.imap_unordered(fetch_page, range(1, total_pages + 1)):
            all_links.extend(links)
        
    # Fetch case details for all links using multiprocessing
    with Pool(8) as pool:
        rows = list(filter(None, pool.map(fetch_case_details, all_links)))

    # Convert the list of dictionaries to a DataFrame
    cases = pd.DataFrame(rows)

    # Save to CSV
    cases.to_csv('./cases.csv', index=False)


In [1]:
cases = pd.read_csv('./cases.csv', index=False)
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87200 entries, 0 to 87199
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   판례정보일련번호  87200 non-null  int64  
 1   사건명       87190 non-null  object 
 2   사건번호      87200 non-null  object 
 3   선고일자      87200 non-null  int64  
 4   법원명       87200 non-null  object 
 5   법원종류코드    87199 non-null  float64
 6   사건종류명     87197 non-null  object 
 7   사건종류코드    87197 non-null  float64
 8   판시사항      73078 non-null  object 
 9   판결요지      63630 non-null  object 
 10  판례내용      87200 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 7.3+ MB


In [3]:
# null 값이 있는지 확인
cases.isnull().sum()

판례정보일련번호        0
사건명            10
사건번호            0
선고일자            0
법원명             0
법원종류코드          1
사건종류명           3
사건종류코드          3
판시사항        14122
판결요지        23570
판례내용            0
dtype: int64

In [7]:
# null 값 치환
cases.fillna('', inplace=True)
cases.isnull().sum()

판례정보일련번호    0
사건명         0
사건번호        0
선고일자        0
법원명         0
법원종류코드      0
사건종류명       0
사건종류코드      0
판시사항        0
판결요지        0
판례내용        0
dtype: int64

In [25]:
import pandas as pd

data_info = pd.read_csv("data_info.csv", encoding="cp949")
data_info.head()

,사고객체,사고장소,사고장소특징,A진행방향,B진행방향,과실비율A,과실비율B,사고유형
0,차대차,직선 도로,추돌 사고,선행자동차(1차사고차량)를 추돌,선행자동차(1차사고차량),80,20,0
1,차대차,직선 도로,추돌 사고,후행 추돌,선행 직진,100,0,1
2,차대차,직선 도로,추돌 사고,주(정)차,후행 추돌,0,100,2
3,차대차,직선 도로,차로 감소 도로 (합류),본선에서 직진,차로 감소 도로에서 본선으로 합류,40,60,3
4,차대차,직선 도로,열린 문 접촉사고,후행직진,선행자동차(정차중 문열림),20,80,4


In [26]:
info = data_info[data_info["사고유형"] == 3]
info.head()

,사고객체,사고장소,사고장소특징,A진행방향,B진행방향,과실비율A,과실비율B,사고유형
3,차대차,직선 도로,차로 감소 도로 (합류),본선에서 직진,차로 감소 도로에서 본선으로 합류,40,60,3


In [30]:
# info['사고장소'] 값 추출

info['사고장소'].unique()[0]

'직선 도로'

### elastic search 

### 데이터 삽입

In [ ]:
import pandas as pd
from elasticsearch import Elasticsearch, helpers
import csv

# Elasticsearch 클라이언트 설정
es = Elasticsearch(
  "https://localhost:9200",
  api_key="SjNPS3FvOEJsQTBjRUJSekYzTDQ6OWhZbGtaUG5SUGFiTWhzd09KaFdYQQ==",
  verify_certs=False
)

# 필드 크기 제한 늘리기
csv.field_size_limit(10**7)  # 제한을 10MB로 증가

# CSV 파일 읽기
df = pd.read_csv('./cases.csv')

def generate_data(df):
    for _, row in df.iterrows():
        yield {
            "_index": "cases",  # 인덱스 이름
            "_source": {
                "판례정보일련번호": row['판례정보일련번호'],
                "사건명": row['사건명'],
                "사건번호": row['사건번호'],
                "선고일자": row['선고일자'],
                "법원명": row['법원명'],
                "법원종류코드": row['법원종류코드'],
                "사건종류명": row['사건종류명'],
                "사건종류코드": row['사건종류코드'],
                "판시사항": row['판시사항'],
                "판결요지": row['판결요지'],
                "판례내용": row['판례내용']
            }
        }

# 데이터 삽입
helpers.bulk(es, generate_data(df))


#### 사건명, 판례내용, 판례제목, 판례번호, 판결일자 검색

In [1]:
from elasticsearch import Elasticsearch, helpers

es = Elasticsearch(
  "https://localhost:9200",
  api_key="SjNPS3FvOEJsQTBjRUJSekYzTDQ6OWhZbGtaUG5SUGFiTWhzd09KaFdYQQ==",
  verify_certs=False
)

/home/taeyoung/.local/lib/python3.10/site-packages/elasticsearch/_sync/client/__init__.py:395: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [2]:
# 복합 검색 쿼리
query = {
    "query": {
        "bool": {
            "should": [
                {"match": {"사건명": "고속도로(자동차전용도로포함)"}},
                {"match": {"사건명": "주행차로와 추월차로"}},
                {"match": {"사건명": "추월차로에서 직진"}},
                {"match": {"사건명": "주행차로에서 추월차로 진로변경"}},
                {"match": {"판례내용": "고속도로(자동차전용도로포함)"}},
                {"match": {"판례내용": "주행차로와 추월차로"}},
                {"match": {"판례내용": "추월차로에서 직진"}},
                {"match": {"판례내용": "주행차로에서 추월차로 진로변경"}},
                {"match": {"판시사항": "고속도로(자동차전용도로포함)"}},
                {"match": {"판시사항": "주행차로와 추월차로"}},
                {"match": {"판시사항": "추월차로에서 직진"}},
                {"match": {"판시사항": "주행차로에서 추월차로 진로변경"}},
                {"match": {"판결요지": "고속도로(자동차전용도로포함)"}},
                {"match": {"판결요지": "주행차로와 추월차로"}},
                {"match": {"판결요지": "추월차로에서 직진"}},
                {"match": {"판결요지": "주행차로에서 추월차로 진로변경"}},
                {"match": {"사건종류명": "고속도로(자동차전용도로포함)"}},
                {"match": {"사건종류명": "주행차로와 추월차로"}},
                {"match": {"사건종류명": "추월차로에서 직진"}},
                {"match": {"사건종류명": "주행차로에서 추월차로 진로변경"}}
            ]
        }
    }
}

# 검색 요청
response = es.search(index="cases", body=query)

# 검색 결과 출력
for hit in response['hits']['hits'][:5]:
    print("Score:", hit['_score'])
    print(hit['_source'])

/tmp/ipykernel_156024/1368967135.py:32: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index="cases", body=query)
/home/taeyoung/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Score: 43.742977
{'판례정보일련번호': 162598, '사건명': '도로교통법위반', '사건번호': '2011도9821', '선고일자': 20120412, '법원명': '대법원', '법원종류코드': 400201.0, '사건종류명': '형사', '사건종류코드': 400102.0, '판시사항': '[1] 교차로에 교통섬이 설치되고 그 오른쪽으로 직진 차로에서 분리된 우회전차로가 설치되어 있는 경우, 우회전차로 아닌 직진 차로를 따라 우회전하는 행위가 \n          구 도로교통법 제25조 제1항에서 정한 ‘교차로 통행방법’에 위배되는지 여부(원칙적 적극)\n        <br/>[2] 자동차 운전자인 피고인이, 교통섬이 설치되고 그 오른쪽으로 직진 차로에서 분리된 우회전차로가 설치되어 있는 교차로에서 우회전차로가 아닌 직진 2개 차로 중 오른쪽 차로를 따라 교차로에 진입하는 방법으로 우회전하였다고 하여 구 도로교통법 위반으로 기소된 사안에서, 피고인의 행위가 \n          같은 법 제25조 제1항에서 정한 ‘교차로 통행방법’에 위배되지 않는다고 본 원심판결에 법리오해의 위법이 있다고 한 사례\n        <br/>', '판결요지': '[1] \n          구 도로교통법(2010. 7. 23. 법률 제10382호로 개정되기 전의 것) 제25조 제1항, \n          제2조 제12호 및 \n          ‘도로의 구조·시설 기준에 관한 규칙’ 제2조 제24호, \n          제43호, \n          제32조 제3항의 내용과 취지 등을 종합하면, 교통섬이 설치되고 그 오른쪽으로 직진 차로에서 분리된 우회전차로가 설치되어 있는 교차로에서 우회전을 하고자 하는 운전자는 특별한 사정이 없는 한 도로 우측 가장자리인 우회전차로를 따라 서행하면서 우회전하여야 하고, 우회전차로가 아닌 직진 차로를 따라 교차로에 진입하는 방법으로 우회전하여서는 아니된다.\n        <br/>[2] 자동차 운전자인 피고인이, 교통